In [1]:
import argparse
import cv2
import numpy as np
import os
import pdb

import torch
import torch.nn as nn
import torch.nn.functional as F
from model.anchors import Anchors, anchor_target, anchors2bboxes
from ops import Voxelization, nms_cuda
from utils import limit_period


In [2]:
def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    '''
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    논문에 표기된 대로 car, pedstrian cyclist를 검출하는 포인터 범위
    '''
    flag_x_low = pts[:, 0] > point_range[0] # 0
    flag_y_low = pts[:, 1] > point_range[1] # -39.68
    flag_z_low = pts[:, 2] > point_range[2] # -3
    flag_x_high = pts[:, 0] < point_range[3] # 69.12
    flag_y_high = pts[:, 1] < point_range[4] # 39.68
    flag_z_high = pts[:, 2] < point_range[5] # 1
    keep_mask = flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    pts = pts[keep_mask]
    return pts 

In [36]:
pc_file_list = sorted(os.listdir("dataset/testing/velodyne/"))

input_pc_path = "dataset/testing/velodyne/" + pc_file_list[3]

suffix = os.path.splitext(input_pc_path)[1]

input_pc = np.fromfile(input_pc_path, dtype=np.float32).reshape(-1, 4)

input_pc_fillered = point_range_filter(input_pc)

pc_torch = torch.from_numpy(input_pc_fillered)


In [37]:
for i, pts in enumerate([pc_torch]):
    # print(pts.new_zeros())
    voxel_layer(pts)

TypeError: 'int' object is not subscriptable

In [13]:
a

tensor([[22.3920,  4.3760,  0.9810,  0.3800],
        [22.4780,  4.4660,  0.9840,  0.2700],
        [21.9480,  4.4320,  0.9660,  0.2100],
        ...,
        [ 3.7410, -1.4020, -1.7470,  0.4700],
        [ 3.7400, -1.3880, -1.7440,  0.0000],
        [ 5.5840, -1.8490, -2.6270,  0.0000]])

In [12]:
for i, pts in enumerate([a]):
    print(i)
    print(pts)

0
tensor([[22.3920,  4.3760,  0.9810,  0.3800],
        [22.4780,  4.4660,  0.9840,  0.2700],
        [21.9480,  4.4320,  0.9660,  0.2100],
        ...,
        [ 3.7410, -1.4020, -1.7470,  0.4700],
        [ 3.7400, -1.3880, -1.7440,  0.0000],
        [ 5.5840, -1.8490, -2.6270,  0.0000]])
